In [1]:
from sys import path
path +=['..']

In [2]:
from mylib import Puller, PlotDisplayer, Slider, Time
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os

output_notebook()
pl = Puller(simulate=True)  #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

In [3]:
Time.SetZeroTime()
Time.unfreez()

In [4]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [5]:
pl.ms.motorM.Set_velocity(v=20, a=20)

0

In [6]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(0);
while pl.ms.IsInMotion():
    pass

In [7]:
pl.tg.SetZeroWeight(3) # калибровка тензодатчика

9275.785413727697

In [8]:
# pl.sim.l0=pl.ms.Distance(useL0=False)+2

In [9]:
pl.tg.ReadValue(tau=0)

0.0

In [10]:
pl.SetW(20, dw=0.1, tau=1,quiet=False); # натяжка волокна

w=  0.0 , dw=  -20.0 , dx=   0.04623303699500446
w=  16.8279816240738 , dw=  -3.1720183759262 , dx=   0.007332602146151499
w=  19.495859861237477 , dw=  -0.5041401387625228 , dx=   0.00116539648430372
w=  19.919848625380837 , dw=  -0.08015137461916311 , dx=   0.00018528207339841147
SetW finish value= 19.987256357677325


In [11]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_min+15); # опускание молибдена вниз перед разогревом
while pl.ms.motorM.IsInMotion():
    pass

In [12]:
x0=60
pl.ms.x0=x0
pl.sim.hotFL=True

In [13]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
# dp2=PlotDisplayer()
# dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
#     'time': pl.times,
#     'kof': pl.trueKmas
# }))
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  motors=pl.data[['time','motorR','motorL','motorM','hFire']]
                #   v_and_a=pl.data[['time','vL','vR','aL','aR']]
         )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=5,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=0.5,value=0.001,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-10, max=23.5,value=0.0,step=0.1)
sl.NewSl('Kp',min=0, max=10,value=0.35,step=0.01)
sl.NewSl('Ki',min=0, max=10,value=0,step=0.01)
sl.NewSl('Kd',min=0, max=10,value=0,step=0.01)
sl.NewSl('T0',min=0, max=200,value=20,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewBtn('start dv','start dv','stop dv')
sl.NewBtn('start stabili','start st','stop st')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')
sl.NewTt('T')

In [ ]:
pl.MotorsControlStart()
t0 = time.time()
t = t0
k = 1
dp.Show()
# dp2.Show()
sl.Display()
pl.Clear()

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()  
        T = sl.Sl['T0']() * k**2
        if pl.PulMotorsControl(sl.Sl['molibdenH'](), T,
                                    upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'],
                                    vsFl=sl.BtnFl['start stabili'], dhKof=0.5, ah=9,Kp=sl.Sl['Kp']()*0.001,Ki=sl.Sl['Ki'](),Kd=sl.Sl['Kd']()
                                 )==-1:
            break
        x=pl.data['x'].iloc[-1]
        pl.v=sl.Sl['v']()
        pl.a=sl.Sl['a']()
        T=sl.Sl['T0']()
        ww=int(sl.Sl['window']())
        k = pl.ms.R_x(x)/pl.ms.R_x(0)
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',pl.dv)
        sl.ChangeValueTt('T',T)
        dp.Apdate(for_all=pl.data.iloc[-ww:])
#         dp2.Apdate(for_all=pd.DataFrame({
#                     'time': pl.times,
#                     'kof': pl.trueKmas
#                 }))
        push_notebook()    
        Time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(2,dw=1,tau=0.5)
print('vse')

start: 0.5555555555555571 0.5450584623548735
move: 0.6737109471304805 0.4851621173100611
stop: 0.5555555555555571 0.5225260968615899
start: 1.0 0.9898586273193359
move: 1.669771964054533 1.5005443639568767
stop: 1.0 0.8994300908855877
start: 1.0 0.9898021221160889
move: 1.8919941862767544 1.8106138189977017
stop: 1.0 0.764577575558981
start: 1.0 0.9898691177368164
move: 1.8919941862767544 1.7978269537633267
stop: 1.0 0.8460683306401577
start: 1.0 0.9898855686187744
move: 1.8911527606492342 1.7357932540421785
stop: 1.0 0.806980034971744
start: 1.0 0.9896864891052246
move: 1.8895386033265424 1.8040030770509077
stop: 1.0 0.8715623192994428
start: 1.0 0.9897172451019287
move: 1.8886215867686573 1.6646244706797901
stop: 1.0 0.930598992316753
start: 1.0 0.9898805618286133
stop: 1.0 0.8153881791337341
start: 1.0 0.9898350238800049
stop: 1.0 0.7966644528611511
move: 1.8884773019059509 1.8108374837144225
start: 1.0 0.9898838996887207
move: 1.8884773019059509 1.8544723752244323
stop: 1.0 0.80813

In [ ]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

In [1]:
pl.ms.Move(60, v=1, a=1, vdiff=0, da=0)

NameError: name 'pl' is not defined

### Для калибровки

In [2]:
pl.Clear()

dp1 = PlotDisplayer()
dp1.CreateMaket(mainParam='time', tension=pl.data[['time', 'tension']])

sl1 = Slider()
sl1.NewBtn('stop', 'stop', 'stopped')
sl1.NewTt('Tension')

dp1.Show()
sl1.Display()

with ui_events() as poll:
    while not sl1.BtnFl['stop']:
        poll(10)
        
        pl.Read(False, False, False)
        
        dp1.Apdate(for_all=pl.data.iloc[-200:])
        sl1.ChangeValueTt('Tension', pl.tg.ReadValue(tau=0))
        Time.sleep(.04)
        push_notebook()

NameError: name 'pl' is not defined

In [47]:
pl.tg.weightCoef = -0.0075585384235655265

In [50]:
pl.tg.weightCoef *= 265598 / 1000 ** 2

In [37]:
pl.tg.weightCoef *= 1000 / 9.8

In [54]:
-0.0075585384235655265 * 265598 / 1000 ** 2

-0.0020075326882221566